In [127]:
import numpy as np
import pandas as pd
pd.set_option('max_columns', None)
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
%matplotlib inline
import copy
import datetime
from sklearn.utils import shuffle
from scipy import stats
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler, LabelEncoder
#import plotly.offline as py
#py.init_notebook_mode(connected=True)
#import plotly.graph_objs as go
#import plotly.tools as tls
import xgboost as xgb
import lightgbm as lgb
#from catboost import CatBoostRegressor, CatBoostClassifier
import optuna
from optuna.visualization import plot_optimization_history
from sklearn import model_selection
from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score, roc_auc_score, log_loss, classification_report, confusion_matrix
import json
import ast
import time
from sklearn import linear_model

import warnings
warnings.filterwarnings('ignore')

import os
import glob
import gc

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import LabelEncoder


In [250]:
path = 'C:\\Users\\bokhy\\Desktop\\kaggle\\google-cloud-ncaa-march-madness-2020-division-1-womens-tournament\\'
SeasonCompactResults = pd.read_csv(os.path.join(path, 'WDataFiles_Stage1\\WRegularSeasonCompactResults.csv'))
SeasonDetailedResults = pd.read_csv(os.path.join(path, 'WDataFiles_Stage1\\WRegularSeasonDetailedResults.csv'))
TourneyCompactResults = pd.read_csv(os.path.join(path, 'WDataFiles_Stage1\\WNCAATourneyCompactResults.csv'))
TourneyDetailedResults = pd.read_csv(os.path.join(path, 'WDataFiles_Stage1\\WNCAATourneyDetailedResults.csv'))
TourneySeeds = pd.read_csv(os.path.join(path, 'WDataFiles_Stage1\\WNCAATourneySeeds.csv'))
GameCities = pd.read_csv(os.path.join(path, 'WDataFiles_Stage1\\WGameCities.csv'))
Cities = pd.read_csv(os.path.join(path, 'WDataFiles_Stage1\\Cities.csv'))
Teams = pd.read_csv(os.path.join(path, 'WDataFiles_Stage1\\WTeams.csv'))
Conferences = pd.read_csv(os.path.join(path, 'WDataFiles_Stage1\\WTeamConferences.csv'))
Seasons = pd.read_csv(os.path.join(path, 'WDataFiles_Stage1\\WSeasons.csv'))
TourneySlots = pd.read_csv(os.path.join(path, 'WDataFiles_Stage1\\WNCAATourneySlots.csv'))

test = pd.read_csv(os.path.join(path, 'WSampleSubmissionStage1_2020.csv'))

In [251]:
# format ID
test = test.drop(['Pred'], axis=1)
test['Season'] = test['ID'].apply(lambda x: int(x.split('_')[0]))
test['WTeamID'] = test['ID'].apply(lambda x: int(x.split('_')[1]))
test['LTeamID'] = test['ID'].apply(lambda x: int(x.split('_')[2]))

In [80]:
TourneyDetailedResults.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2010,11,3103,63,3237,49,H,0,23,54,5,9,12,19,10,26,14,18,7,0,15,20,54,3,13,6,10,11,27,11,23,7,6,19
1,2010,11,3104,73,3399,68,N,0,26,62,5,12,16,28,16,31,15,20,5,2,25,25,63,4,21,14,27,14,26,7,20,4,2,27
2,2010,11,3110,71,3224,59,A,0,29,62,6,15,7,12,14,23,18,13,6,2,17,19,58,2,14,19,23,17,23,8,15,6,0,15
3,2010,11,3111,63,3267,58,A,0,27,52,4,11,5,9,6,40,14,27,5,10,18,18,74,6,26,16,25,22,22,15,11,14,5,14
4,2010,11,3119,74,3447,70,H,1,30,74,7,20,7,11,14,33,18,11,5,3,18,25,74,9,17,11,21,21,32,12,14,4,2,14


In [252]:
SeasonDetailedResults_1 = SeasonDetailedResults[[
    'Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc', 'NumOT', 
    'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 
    'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF']]

In [253]:
SeasonDetailedResults_2 = SeasonDetailedResults[[
    'Season', 'DayNum', 'LTeamID', 'LScore', 'WTeamID', 'WScore', 'WLoc', 'NumOT', 
    'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF', 
    'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF']]

In [254]:
SeasonDetailedResults_2.loc[SeasonDetailedResults_1['WLoc'] == 'H', 'WLoc'] = 'A'
SeasonDetailedResults_2.loc[SeasonDetailedResults_1['WLoc'] == 'A', 'WLoc'] = 'H'
SeasonDetailedResults_1.columns.values[6] = 'location'
SeasonDetailedResults_2.columns.values[6] = 'location'

In [255]:
SeasonDetailedResults_1.columns = [x.replace('W','T1_').replace('L','T2_') for x in list(SeasonDetailedResults_1.columns)]
SeasonDetailedResults_2.columns = [x.replace('L','T1_').replace('W','T2_') for x in list(SeasonDetailedResults_2.columns)]

In [256]:
SeasonDetailedResults_1.head()

,Season,DayNum,T1_TeamID,T1_Score,T2_TeamID,T2_Score,location,NumOT,T1_FGM,T1_FGA,T1_FGM3,T1_FGA3,T1_FTM,T1_FTA,T1_OR,T1_DR,T1_Ast,T1_TO,T1_Stl,T1_Blk,T1_PF,T2_FGM,T2_FGA,T2_FGM3,T2_FGA3,T2_FTM,T2_FTA,T2_OR,T2_DR,T2_Ast,T2_TO,T2_Stl,T2_Blk,T2_PF
0,2010,11,3103,63,3237,49,H,0,23,54,5,9,12,19,10,26,14,18,7,0,15,20,54,3,13,6,10,11,27,11,23,7,6,19
1,2010,11,3104,73,3399,68,N,0,26,62,5,12,16,28,16,31,15,20,5,2,25,25,63,4,21,14,27,14,26,7,20,4,2,27
2,2010,11,3110,71,3224,59,A,0,29,62,6,15,7,12,14,23,18,13,6,2,17,19,58,2,14,19,23,17,23,8,15,6,0,15
3,2010,11,3111,63,3267,58,A,0,27,52,4,11,5,9,6,40,14,27,5,10,18,18,74,6,26,16,25,22,22,15,11,14,5,14
4,2010,11,3119,74,3447,70,H,1,30,74,7,20,7,11,14,33,18,11,5,3,18,25,74,9,17,11,21,21,32,12,14,4,2,14


In [257]:
SeasonDetailedResults = pd.concat([SeasonDetailedResults_1, SeasonDetailedResults_2]).sort_values(by=['DayNum']).reset_index(drop = True)

In [258]:
SeasonDetailedResults.head()

,Season,DayNum,T1_TeamID,T1_Score,T2_TeamID,T2_Score,location,NumOT,T1_FGM,T1_FGA,T1_FGM3,T1_FGA3,T1_FTM,T1_FTA,T1_OR,T1_DR,T1_Ast,T1_TO,T1_Stl,T1_Blk,T1_PF,T2_FGM,T2_FGA,T2_FGM3,T2_FGA3,T2_FTM,T2_FTA,T2_OR,T2_DR,T2_Ast,T2_TO,T2_Stl,T2_Blk,T2_PF
0,2019,1,3113,81,3230,43,H,0,29,65,10,28,13,23,19,39,23,19,11,8,20,17,66,3,17,6,15,13,21,5,16,12,1,21
1,2019,1,3442,64,3281,89,H,0,17,59,4,20,26,33,6,26,3,10,6,1,19,33,71,9,27,14,17,13,39,18,14,7,7,25
2,2019,1,3189,69,3314,100,H,0,27,58,1,12,14,22,7,26,12,23,7,1,20,35,67,11,24,19,29,12,30,23,21,14,4,21
3,2019,1,3201,73,3319,86,A,0,28,71,5,20,12,17,16,15,12,14,17,1,16,32,56,15,20,7,8,8,31,21,23,6,5,16
4,2019,1,3453,55,3321,57,H,0,20,50,8,14,7,11,13,28,17,21,9,1,18,19,53,5,19,14,20,9,18,8,16,10,4,16


In [259]:
TourneyDetailedResults.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2010,138,3124,69,3201,55,N,0,28,57,1,5,12,19,13,24,22,12,6,2,12,21,61,10,34,3,5,17,19,12,18,4,1,18
1,2010,138,3173,67,3395,66,N,0,23,59,9,26,12,19,13,34,13,16,3,10,14,22,73,8,27,14,15,18,26,8,8,8,6,22
2,2010,138,3181,72,3214,37,H,0,26,57,4,13,16,22,13,34,15,11,10,7,11,15,56,4,15,3,8,10,21,4,16,6,4,20
3,2010,138,3199,75,3256,61,H,0,25,63,3,15,22,26,20,27,13,17,8,3,21,21,62,2,20,17,22,16,21,13,16,5,4,24
4,2010,138,3207,62,3265,42,N,0,24,68,8,25,6,8,20,29,16,8,5,5,18,13,60,5,26,11,17,16,22,9,10,3,4,12


In [260]:
TourneyDetailedResults.shape

(630, 34)

In [261]:
TourneyDetailedResults_1 = TourneyDetailedResults[[
    'Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc', 'NumOT', 
    'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 
    'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF']]

In [262]:
TourneyDetailedResults_2 = TourneyDetailedResults[[
    'Season', 'DayNum', 'LTeamID', 'LScore', 'WTeamID', 'WScore', 'WLoc', 'NumOT', 
    'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF', 
    'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF']]

In [263]:
TourneyDetailedResults_2.loc[TourneyDetailedResults_1['WLoc'] == 'H', 'WLoc'] = 'A'
TourneyDetailedResults_2.loc[TourneyDetailedResults_1['WLoc'] == 'A', 'WLoc'] = 'H'
TourneyDetailedResults_1.columns.values[6] = 'location'
TourneyDetailedResults_2.columns.values[6] = 'location'

In [264]:
TourneyDetailedResults_1.columns = [x.replace('W','T1_').replace('L','T2_') for x in list(TourneyDetailedResults_1.columns)]
TourneyDetailedResults_2.columns = [x.replace('L','T1_').replace('W','T2_') for x in list(TourneyDetailedResults_2.columns)]

In [265]:
TourneyDetailedResults_1["result"] = 1
TourneyDetailedResults_2["result"] = 0

In [266]:
TourneyDetailedResults = pd.concat([TourneyDetailedResults_1, TourneyDetailedResults_2]).sort_values(by=['DayNum']).reset_index(drop = True)

In [267]:
TourneyDetailedResults.head()

,Season,DayNum,T1_TeamID,T1_Score,T2_TeamID,T2_Score,location,NumOT,T1_FGM,T1_FGA,T1_FGM3,T1_FGA3,T1_FTM,T1_FTA,T1_OR,T1_DR,T1_Ast,T1_TO,T1_Stl,T1_Blk,T1_PF,T2_FGM,T2_FGA,T2_FGM3,T2_FGA3,T2_FTM,T2_FTA,T2_OR,T2_DR,T2_Ast,T2_TO,T2_Stl,T2_Blk,T2_PF,result
0,2019,137,3125,52,3376,74,N,0,18,52,7,25,9,11,10,12,11,17,3,2,13,31,56,5,15,7,11,14,24,14,14,6,6,16,0
1,2019,137,3332,78,3340,40,H,0,29,63,8,23,12,15,20,30,17,12,16,2,8,12,49,8,28,8,11,8,15,10,17,5,2,11,1
2,2019,137,3281,77,3179,76,N,1,27,58,9,25,14,16,11,26,15,19,10,5,16,28,65,12,26,8,11,13,21,23,16,8,3,20,1
3,2019,137,3280,103,3380,46,H,0,36,68,3,13,28,37,17,34,13,13,17,3,12,15,55,5,16,11,12,7,21,6,28,6,2,27,1
4,2019,137,3276,84,3243,54,N,0,35,63,5,13,9,14,17,33,22,21,9,2,17,20,55,7,22,7,17,6,13,11,17,15,4,11,1


In [268]:
TourneyDetailedResults.shape

(1260, 35)

In [269]:
SeasonDetailedResults.loc[SeasonDetailedResults.location=='N','location'] = '0'
SeasonDetailedResults.loc[SeasonDetailedResults.location=='H','location'] = '1'
SeasonDetailedResults.loc[SeasonDetailedResults.location=='A','location'] = '-1'
SeasonDetailedResults.location = SeasonDetailedResults.location.astype(int) 
SeasonDetailedResults['PointDiff'] = SeasonDetailedResults['T1_Score'] - SeasonDetailedResults['T2_Score']

In [270]:
TourneyDetailedResults.loc[TourneyDetailedResults.location=='N','location'] = '0'
TourneyDetailedResults.loc[TourneyDetailedResults.location=='H','location'] = '1'
TourneyDetailedResults.loc[TourneyDetailedResults.location=='A','location'] = '-1'
TourneyDetailedResults.location = TourneyDetailedResults.location.astype(int) 
TourneyDetailedResults['PointDiff'] = TourneyDetailedResults['T1_Score'] - TourneyDetailedResults['T2_Score']

In [271]:
SeasonDetailedResults.shape

(103164, 35)

In [272]:
TourneyDetailedResults.shape

(1260, 36)

# Season Data 

In [273]:
boxscore_cols = [
        'T1_FGM', 'T1_FGA', 'T1_FGM3', 'T1_FGA3', 'T1_FTM', 'T1_FTA', 'T1_OR', 'T1_DR', 'T1_Ast', 'T1_TO', 'T1_Stl', 'T1_Blk', 'T1_PF', 
        'T2_FGM', 'T2_FGA', 'T2_FGM3', 'T2_FGA3', 'T2_FTM', 'T2_FTA', 'T2_OR', 'T2_DR', 'T2_Ast', 'T2_TO', 'T2_Stl', 'T2_Blk', 'T2_PF', 
        'PointDiff']

In [274]:
SeasonDetailedResults_Stat = SeasonDetailedResults.groupby(['Season', 'T1_TeamID'])[boxscore_cols].agg([np.mean]).reset_index()
#SeasonDetailedResults_Stat = SeasonDetailedResults.groupby(['Season', 'T1_TeamID'])[boxscore_cols].agg(['median','mean','sum', 'count', 'var','std']).reset_index()
SeasonDetailedResults_Stat.columns = [' '.join(col).strip() for col in SeasonDetailedResults_Stat.columns.values]

In [275]:
SeasonDetailedResults_Stat.head()

,Season,T1_TeamID,T1_FGM mean,T1_FGA mean,T1_FGM3 mean,T1_FGA3 mean,T1_FTM mean,T1_FTA mean,T1_OR mean,T1_DR mean,T1_Ast mean,T1_TO mean,T1_Stl mean,T1_Blk mean,T1_PF mean,T2_FGM mean,T2_FGA mean,T2_FGM3 mean,T2_FGA3 mean,T2_FTM mean,T2_FTA mean,T2_OR mean,T2_DR mean,T2_Ast mean,T2_TO mean,T2_Stl mean,T2_Blk mean,T2_PF mean,PointDiff mean
0,2010,3102,19.142857,53.142857,4.571429,16.071429,8.964286,13.214286,12.464286,19.535714,10.250000,17.714286,6.035714,0.642857,14.964286,27.000000,56.392857,7.714286,20.500000,10.071429,14.892857,12.821429,23.857143,19.535714,14.392857,9.821429,5.035714,13.607143,-19.964286
1,2010,3103,22.233333,55.266667,3.933333,12.466667,13.833333,19.600000,14.200000,23.566667,13.433333,18.266667,8.033333,2.400000,17.533333,20.966667,54.966667,4.833333,15.166667,12.866667,18.833333,13.633333,21.600000,11.700000,18.700000,9.233333,3.800000,17.633333,2.600000
2,2010,3104,24.724138,62.103448,4.344828,14.724138,10.344828,16.793103,14.482759,25.931034,13.103448,19.172414,7.379310,2.931034,20.413793,23.896552,60.137931,5.103448,16.517241,14.103448,21.965517,14.862069,26.482759,12.517241,19.241379,8.758621,3.586207,16.413793,-2.862069
3,2010,3105,20.370370,50.851852,3.037037,9.888889,17.000000,24.333333,13.666667,23.370370,9.740741,25.037037,8.777778,5.148148,19.148148,22.777778,57.518519,3.740741,12.777778,15.444444,23.518519,15.185185,21.296296,13.481481,20.740741,12.222222,4.407407,20.962963,-3.962963
4,2010,3106,18.448276,53.000000,2.689655,9.586207,15.551724,24.000000,15.000000,24.758621,11.482759,20.103448,8.034483,4.482759,22.068966,17.896552,50.689655,3.724138,11.827586,17.896552,26.655172,12.379310,23.413793,9.517241,18.310345,8.793103,3.965517,20.068966,-2.275862


In [276]:
SeasonDetailedResults_Stat_T1 = SeasonDetailedResults_Stat.copy()
SeasonDetailedResults_Stat_T2 = SeasonDetailedResults_Stat.copy()

SeasonDetailedResults_Stat_T1.columns = ["T1_" + x.replace("T1_","").replace("T2_","opponent_") for x in list(SeasonDetailedResults_Stat_T1.columns)]
SeasonDetailedResults_Stat_T2.columns = ["T2_" + x.replace("T1_","").replace("T2_","opponent_") for x in list(SeasonDetailedResults_Stat_T2.columns)]

SeasonDetailedResults_Stat_T1.columns.values[0] = "Season"
SeasonDetailedResults_Stat_T2.columns.values[0] = "Season"

# Tournament Data

In [277]:
TourneyDetailedResults = TourneyDetailedResults[['Season', 'DayNum', 'T1_TeamID', 'T1_Score', 'T2_TeamID' ,'T2_Score','result']]

In [278]:
TourneyDetailedResults = pd.merge(TourneyDetailedResults, SeasonDetailedResults_Stat_T1, on = ['Season', 'T1_TeamID'], how = 'left')
TourneyDetailedResults = pd.merge(TourneyDetailedResults, SeasonDetailedResults_Stat_T2, on = ['Season', 'T2_TeamID'], how = 'left')

In [279]:
TourneyDetailedResults.head()

,Season,DayNum,T1_TeamID,T1_Score,T2_TeamID,T2_Score,result,T1_FGM mean,T1_FGA mean,T1_FGM3 mean,T1_FGA3 mean,T1_FTM mean,T1_FTA mean,T1_OR mean,T1_DR mean,T1_Ast mean,T1_TO mean,T1_Stl mean,T1_Blk mean,T1_PF mean,T1_opponent_FGM mean,T1_opponent_FGA mean,T1_opponent_FGM3 mean,T1_opponent_FGA3 mean,T1_opponent_FTM mean,T1_opponent_FTA mean,T1_opponent_OR mean,T1_opponent_DR mean,T1_opponent_Ast mean,T1_opponent_TO mean,T1_opponent_Stl mean,T1_opponent_Blk mean,T1_opponent_PF mean,T1_PointDiff mean,T2_FGM mean,T2_FGA mean,T2_FGM3 mean,T2_FGA3 mean,T2_FTM mean,T2_FTA mean,T2_OR mean,T2_DR mean,T2_Ast mean,T2_TO mean,T2_Stl mean,T2_Blk mean,T2_PF mean,T2_opponent_FGM mean,T2_opponent_FGA mean,T2_opponent_FGM3 mean,T2_opponent_FGA3 mean,T2_opponent_FTM mean,T2_opponent_FTA mean,T2_opponent_OR mean,T2_opponent_DR mean,T2_opponent_Ast mean,T2_opponent_TO mean,T2_opponent_Stl mean,T2_opponent_Blk mean,T2_opponent_PF mean,T2_PointDiff mean
0,2019,137,3125,52,3376,74,0,26.968750,59.687500,10.43750,29.093750,11.312500,16.468750,11.812500,29.156250,16.312500,12.562500,5.562500,2.125000,14.656250,22.000000,60.437500,6.250000,20.750000,9.343750,13.46875,10.96875,23.312500,11.593750,12.031250,5.625000,2.875000,17.531250,16.093750,27.433333,63.233333,6.000000,18.233333,14.866667,20.533333,13.833333,25.166667,13.100000,13.600000,8.333333,6.300000,16.766667,24.466667,62.133333,4.800000,15.533333,12.766667,17.233333,13.466667,23.300000,11.033333,15.866667,6.766667,3.133333,18.900000,9.233333
1,2019,137,3332,78,3340,40,1,31.937500,63.625000,9.75000,23.343750,11.531250,15.062500,10.937500,26.343750,19.031250,10.125000,6.906250,2.593750,13.000000,24.218750,60.281250,6.656250,21.562500,8.656250,11.68750,10.03125,20.750000,13.687500,13.218750,4.687500,2.500000,14.875000,21.406250,25.967742,60.258065,5.838710,16.806452,12.967742,16.225806,11.064516,28.258065,17.225806,14.741935,9.580645,5.258065,14.935484,22.032258,61.870968,6.548387,21.612903,9.516129,14.290323,13.354839,24.258065,14.548387,15.903226,7.548387,1.806452,16.225806,10.612903
2,2019,137,3281,77,3179,76,1,22.757576,50.757576,8.30303,22.878788,12.151515,16.424242,9.242424,26.393939,13.969697,16.181818,5.939394,3.181818,17.181818,21.030303,57.787879,4.848485,17.030303,11.030303,15.69697,12.30303,20.242424,8.393939,12.212121,6.757576,1.939394,18.909091,8.030303,28.406250,59.937500,8.750000,24.437500,14.781250,19.343750,11.093750,28.562500,21.093750,17.781250,9.031250,2.562500,17.156250,24.312500,63.156250,7.187500,25.125000,11.250000,15.750000,11.468750,22.281250,13.593750,16.281250,8.656250,2.875000,19.562500,13.281250
3,2019,137,3280,103,3380,46,1,32.531250,66.156250,5.25000,13.781250,15.781250,21.531250,17.343750,25.468750,16.562500,12.625000,8.656250,5.437500,16.937500,21.093750,55.625000,5.187500,15.562500,10.375000,14.93750,11.03125,18.843750,9.281250,19.656250,5.937500,3.937500,19.968750,28.343750,21.214286,55.750000,4.428571,16.178571,11.750000,18.964286,12.750000,22.857143,11.464286,18.392857,8.642857,2.000000,22.392857,21.071429,50.607143,3.857143,14.392857,15.464286,25.142857,11.535714,24.714286,11.500000,19.321429,8.214286,3.107143,19.285714,-2.857143
4,2019,137,3276,84,3243,54,1,27.093750,60.437500,4.56250,13.937500,13.218750,18.875000,13.750000,25.656250,15.187500,16.250000,7.312500,3.531250,17.156250,23.156250,57.562500,5.875000,18.875000,11.468750,16.84375,10.53125,21.937500,12.687500,16.625000,7.968750,3.437500,17.750000,8.312500,23.062500,56.718750,5.656250,18.781250,13.031250,18.062500,10.500000,24.437500,13.000000,15.062500,8.000000,4.468750,14.562500,23.031250,60.093750,6.343750,20.156250,11.406250,15.187500,13.375000,24.750000,13.906250,15.937500,8.218750,2.812500,16.500000,1.000000


In [280]:
last14days_stats_T1 = SeasonDetailedResults.loc[SeasonDetailedResults.DayNum>118].reset_index(drop=True)
last14days_stats_T1['last14days'] = np.where(last14days_stats_T1['T1_Score'] > last14days_stats_T1['T2_Score'], 1, 0)
last14days_stats_T1 = last14days_stats_T1.groupby(['Season','T1_TeamID'])['last14days'].mean().reset_index(name='T1_win_ratio_14d')

In [281]:
last14days_stats_T2 = SeasonDetailedResults.loc[SeasonDetailedResults.DayNum>118].reset_index(drop=True)
last14days_stats_T2['last14days'] = np.where(last14days_stats_T2['T1_Score'] > last14days_stats_T2['T2_Score'], 1, 0)
last14days_stats_T2 = last14days_stats_T2.groupby(['Season','T2_TeamID'])['last14days'].mean().reset_index(name='T2_win_ratio_14d')

In [282]:
TourneyDetailedResults = pd.merge(TourneyDetailedResults, last14days_stats_T1, on = ['Season', 'T1_TeamID'], how = 'left')
TourneyDetailedResults = pd.merge(TourneyDetailedResults, last14days_stats_T2, on = ['Season', 'T2_TeamID'], how = 'left')

In [283]:
TourneyDetailedResults.head()

,Season,DayNum,T1_TeamID,T1_Score,T2_TeamID,T2_Score,result,T1_FGM mean,T1_FGA mean,T1_FGM3 mean,T1_FGA3 mean,T1_FTM mean,T1_FTA mean,T1_OR mean,T1_DR mean,T1_Ast mean,T1_TO mean,T1_Stl mean,T1_Blk mean,T1_PF mean,T1_opponent_FGM mean,T1_opponent_FGA mean,T1_opponent_FGM3 mean,T1_opponent_FGA3 mean,T1_opponent_FTM mean,T1_opponent_FTA mean,T1_opponent_OR mean,T1_opponent_DR mean,T1_opponent_Ast mean,T1_opponent_TO mean,T1_opponent_Stl mean,T1_opponent_Blk mean,T1_opponent_PF mean,T1_PointDiff mean,T2_FGM mean,T2_FGA mean,T2_FGM3 mean,T2_FGA3 mean,T2_FTM mean,T2_FTA mean,T2_OR mean,T2_DR mean,T2_Ast mean,T2_TO mean,T2_Stl mean,T2_Blk mean,T2_PF mean,T2_opponent_FGM mean,T2_opponent_FGA mean,T2_opponent_FGM3 mean,T2_opponent_FGA3 mean,T2_opponent_FTM mean,T2_opponent_FTA mean,T2_opponent_OR mean,T2_opponent_DR mean,T2_opponent_Ast mean,T2_opponent_TO mean,T2_opponent_Stl mean,T2_opponent_Blk mean,T2_opponent_PF mean,T2_PointDiff mean,T1_win_ratio_14d,T2_win_ratio_14d
0,2019,137,3125,52,3376,74,0,26.968750,59.687500,10.43750,29.093750,11.312500,16.468750,11.812500,29.156250,16.312500,12.562500,5.562500,2.125000,14.656250,22.000000,60.437500,6.250000,20.750000,9.343750,13.46875,10.96875,23.312500,11.593750,12.031250,5.625000,2.875000,17.531250,16.093750,27.433333,63.233333,6.000000,18.233333,14.866667,20.533333,13.833333,25.166667,13.100000,13.600000,8.333333,6.300000,16.766667,24.466667,62.133333,4.800000,15.533333,12.766667,17.233333,13.466667,23.300000,11.033333,15.866667,6.766667,3.133333,18.900000,9.233333,1.000000,1.000000
1,2019,137,3332,78,3340,40,1,31.937500,63.625000,9.75000,23.343750,11.531250,15.062500,10.937500,26.343750,19.031250,10.125000,6.906250,2.593750,13.000000,24.218750,60.281250,6.656250,21.562500,8.656250,11.68750,10.03125,20.750000,13.687500,13.218750,4.687500,2.500000,14.875000,21.406250,25.967742,60.258065,5.838710,16.806452,12.967742,16.225806,11.064516,28.258065,17.225806,14.741935,9.580645,5.258065,14.935484,22.032258,61.870968,6.548387,21.612903,9.516129,14.290323,13.354839,24.258065,14.548387,15.903226,7.548387,1.806452,16.225806,10.612903,0.666667,0.200000
2,2019,137,3281,77,3179,76,1,22.757576,50.757576,8.30303,22.878788,12.151515,16.424242,9.242424,26.393939,13.969697,16.181818,5.939394,3.181818,17.181818,21.030303,57.787879,4.848485,17.030303,11.030303,15.69697,12.30303,20.242424,8.393939,12.212121,6.757576,1.939394,18.909091,8.030303,28.406250,59.937500,8.750000,24.437500,14.781250,19.343750,11.093750,28.562500,21.093750,17.781250,9.031250,2.562500,17.156250,24.312500,63.156250,7.187500,25.125000,11.250000,15.750000,11.468750,22.281250,13.593750,16.281250,8.656250,2.875000,19.562500,13.281250,0.666667,0.200000
3,2019,137,3280,103,3380,46,1,32.531250,66.156250,5.25000,13.781250,15.781250,21.531250,17.343750,25.468750,16.562500,12.625000,8.656250,5.437500,16.937500,21.093750,55.625000,5.187500,15.562500,10.375000,14.93750,11.03125,18.843750,9.281250,19.656250,5.937500,3.937500,19.968750,28.343750,21.214286,55.750000,4.428571,16.178571,11.750000,18.964286,12.750000,22.857143,11.464286,18.392857,8.642857,2.000000,22.392857,21.071429,50.607143,3.857143,14.392857,15.464286,25.142857,11.535714,24.714286,11.500000,19.321429,8.214286,3.107143,19.285714,-2.857143,1.000000,0.200000
4,2019,137,3276,84,3243,54,1,27.093750,60.437500,4.56250,13.937500,13.218750,18.875000,13.750000,25.656250,15.187500,16.250000,7.312500,3.531250,17.156250,23.156250,57.562500,5.875000,18.875000,11.468750,16.84375,10.53125,21.937500,12.687500,16.625000,7.968750,3.437500,17.750000,8.312500,23.062500,56.718750,5.656250,18.781250,13.031250,18.062500,10.500000,24.437500,13.000000,15.062500,8.000000,4.468750,14.562500,23.031250,60.093750,6.343750,20.156250,11.406250,15.187500,13.375000,24.750000,13.906250,15.937500,8.218750,2.812500,16.500000,1.000000,0.500000,0.333333


# Team Quality?

In [284]:
regular_season_effects = SeasonDetailedResults[['Season','T1_TeamID','T2_TeamID','PointDiff']].copy()
regular_season_effects['T1_TeamID'] = regular_season_effects['T1_TeamID'].astype(str)
regular_season_effects['T2_TeamID'] = regular_season_effects['T2_TeamID'].astype(str)
regular_season_effects['win'] = np.where(regular_season_effects['PointDiff']>0,1,0)

In [285]:
regular_season_effects.head()

,Season,T1_TeamID,T2_TeamID,PointDiff,win
0,2019,3113,3230,38,1
1,2019,3442,3281,-25,0
2,2019,3189,3314,-31,0
3,2019,3201,3319,-13,0
4,2019,3453,3321,-2,0


In [286]:
TourneySeeds.head()

,Season,Seed,TeamID
0,1998,W01,3330
1,1998,W02,3163
2,1998,W03,3112
3,1998,W04,3301
4,1998,W05,3272


In [287]:
march_madness = pd.merge(TourneySeeds[['Season','TeamID']],TourneySeeds[['Season','TeamID']],on='Season')
march_madness.columns = ['Season', 'T1_TeamID', 'T2_TeamID']
march_madness.T1_TeamID = march_madness.T1_TeamID.astype(str)
march_madness.T2_TeamID = march_madness.T2_TeamID.astype(str)
regular_season_effects = pd.merge(regular_season_effects, march_madness, on = ['Season','T1_TeamID','T2_TeamID'])

In [288]:
regular_season_effects.head()

,Season,T1_TeamID,T2_TeamID,PointDiff,win
0,2019,3355,3266,-39,0
1,2019,3266,3355,39,1
2,2019,3413,3390,-28,0
3,2019,3125,3301,-15,0
4,2019,3349,3401,-11,0


In [289]:
import statsmodels.api as sm

In [290]:
def team_quality(season):
    formula = 'win~-1+T1_TeamID+T2_TeamID'
    glm = sm.GLM.from_formula(formula=formula, 
                              data=regular_season_effects.loc[regular_season_effects.Season==season,:], 
                              family=sm.families.Binomial()).fit()
    
    quality = pd.DataFrame(glm.params).reset_index()
    quality.columns = ['TeamID','quality']
    quality['Season'] = season
    quality['quality'] = np.exp(quality['quality'])
    quality = quality.loc[quality.TeamID.str.contains('T1_')].reset_index(drop=True)
    quality['TeamID'] = quality['TeamID'].apply(lambda x: x[10:14]).astype(int)
    return quality

In [291]:
team_quality

<function __main__.team_quality(season)>

In [292]:
glm_quality = pd.concat([team_quality(2010),
                         team_quality(2011),
                         team_quality(2012),
                         team_quality(2013),
                         team_quality(2014),
                         team_quality(2015),
                         team_quality(2016),
                         team_quality(2017),
                         team_quality(2018)]).reset_index(drop=True)

In [293]:
glm_quality.head()

,TeamID,quality,Season
0,3114,1.000000e+00,2010
1,3122,2.700316e-49,2010
2,3124,9.290078e+00,2010
3,3132,2.818303e+00,2010
4,3151,7.398088e-33,2010


In [294]:
glm_quality_T1 = glm_quality.copy()
glm_quality_T2 = glm_quality.copy()
glm_quality_T1.columns = ['T1_TeamID','T1_quality','Season']
glm_quality_T2.columns = ['T2_TeamID','T2_quality','Season']

In [295]:
TourneyDetailedResults = pd.merge(TourneyDetailedResults, glm_quality_T1, on = ['Season', 'T1_TeamID'], how = 'left')
TourneyDetailedResults = pd.merge(TourneyDetailedResults, glm_quality_T2, on = ['Season', 'T2_TeamID'], how = 'left')

In [296]:
TourneyDetailedResults.head()

,Season,DayNum,T1_TeamID,T1_Score,T2_TeamID,T2_Score,result,T1_FGM mean,T1_FGA mean,T1_FGM3 mean,T1_FGA3 mean,T1_FTM mean,T1_FTA mean,T1_OR mean,T1_DR mean,T1_Ast mean,T1_TO mean,T1_Stl mean,T1_Blk mean,T1_PF mean,T1_opponent_FGM mean,T1_opponent_FGA mean,T1_opponent_FGM3 mean,T1_opponent_FGA3 mean,T1_opponent_FTM mean,T1_opponent_FTA mean,T1_opponent_OR mean,T1_opponent_DR mean,T1_opponent_Ast mean,T1_opponent_TO mean,T1_opponent_Stl mean,T1_opponent_Blk mean,T1_opponent_PF mean,T1_PointDiff mean,T2_FGM mean,T2_FGA mean,T2_FGM3 mean,T2_FGA3 mean,T2_FTM mean,T2_FTA mean,T2_OR mean,T2_DR mean,T2_Ast mean,T2_TO mean,T2_Stl mean,T2_Blk mean,T2_PF mean,T2_opponent_FGM mean,T2_opponent_FGA mean,T2_opponent_FGM3 mean,T2_opponent_FGA3 mean,T2_opponent_FTM mean,T2_opponent_FTA mean,T2_opponent_OR mean,T2_opponent_DR mean,T2_opponent_Ast mean,T2_opponent_TO mean,T2_opponent_Stl mean,T2_opponent_Blk mean,T2_opponent_PF mean,T2_PointDiff mean,T1_win_ratio_14d,T2_win_ratio_14d,T1_quality,T2_quality
0,2019,137,3125,52,3376,74,0,26.968750,59.687500,10.43750,29.093750,11.312500,16.468750,11.812500,29.156250,16.312500,12.562500,5.562500,2.125000,14.656250,22.000000,60.437500,6.250000,20.750000,9.343750,13.46875,10.96875,23.312500,11.593750,12.031250,5.625000,2.875000,17.531250,16.093750,27.433333,63.233333,6.000000,18.233333,14.866667,20.533333,13.833333,25.166667,13.100000,13.600000,8.333333,6.300000,16.766667,24.466667,62.133333,4.800000,15.533333,12.766667,17.233333,13.466667,23.300000,11.033333,15.866667,6.766667,3.133333,18.900000,9.233333,1.000000,1.000000,NaN,NaN
1,2019,137,3332,78,3340,40,1,31.937500,63.625000,9.75000,23.343750,11.531250,15.062500,10.937500,26.343750,19.031250,10.125000,6.906250,2.593750,13.000000,24.218750,60.281250,6.656250,21.562500,8.656250,11.68750,10.03125,20.750000,13.687500,13.218750,4.687500,2.500000,14.875000,21.406250,25.967742,60.258065,5.838710,16.806452,12.967742,16.225806,11.064516,28.258065,17.225806,14.741935,9.580645,5.258065,14.935484,22.032258,61.870968,6.548387,21.612903,9.516129,14.290323,13.354839,24.258065,14.548387,15.903226,7.548387,1.806452,16.225806,10.612903,0.666667,0.200000,NaN,NaN
2,2019,137,3281,77,3179,76,1,22.757576,50.757576,8.30303,22.878788,12.151515,16.424242,9.242424,26.393939,13.969697,16.181818,5.939394,3.181818,17.181818,21.030303,57.787879,4.848485,17.030303,11.030303,15.69697,12.30303,20.242424,8.393939,12.212121,6.757576,1.939394,18.909091,8.030303,28.406250,59.937500,8.750000,24.437500,14.781250,19.343750,11.093750,28.562500,21.093750,17.781250,9.031250,2.562500,17.156250,24.312500,63.156250,7.187500,25.125000,11.250000,15.750000,11.468750,22.281250,13.593750,16.281250,8.656250,2.875000,19.562500,13.281250,0.666667,0.200000,NaN,NaN
3,2019,137,3280,103,3380,46,1,32.531250,66.156250,5.25000,13.781250,15.781250,21.531250,17.343750,25.468750,16.562500,12.625000,8.656250,5.437500,16.937500,21.093750,55.625000,5.187500,15.562500,10.375000,14.93750,11.03125,18.843750,9.281250,19.656250,5.937500,3.937500,19.968750,28.343750,21.214286,55.750000,4.428571,16.178571,11.750000,18.964286,12.750000,22.857143,11.464286,18.392857,8.642857,2.000000,22.392857,21.071429,50.607143,3.857143,14.392857,15.464286,25.142857,11.535714,24.714286,11.500000,19.321429,8.214286,3.107143,19.285714,-2.857143,1.000000,0.200000,NaN,NaN
4,2019,137,3276,84,3243,54,1,27.093750,60.437500,4.56250,13.937500,13.218750,18.875000,13.750000,25.656250,15.187500,16.250000,7.312500,3.531250,17.156250,23.156250,57.562500,5.875000,18.875000,11.468750,16.84375,10.53125,21.937500,12.687500,16.625000,7.968750,3.437500,17.750000,8.312500,23.062500,56.718750,5.656250,18.781250,13.031250,18.062500,10.500000,24.437500,13.000000,15.062500,8.000000,4.468750,14.562500,23.031250,60.093750,6.343750,20.156250,11.406250,15.187500,13.375000,24.750000,13.906250,15.937500,8.218750,2.812500,16.500000,1.000000,0.500000,0.333333,NaN,NaN


In [297]:
TourneySeeds['seed'] = TourneySeeds['Seed'].apply(lambda x: int(x[1:3]))
TourneySeeds.head()

,Season,Seed,TeamID,seed
0,1998,W01,3330,1
1,1998,W02,3163,2
2,1998,W03,3112,3
3,1998,W04,3301,4
4,1998,W05,3272,5


In [298]:
TourneySeeds_T1 = TourneySeeds[['Season','TeamID','seed']].copy()
TourneySeeds_T2 = TourneySeeds[['Season','TeamID','seed']].copy()
TourneySeeds_T1.columns = ['Season','T1_TeamID','T1_seed']
TourneySeeds_T2.columns = ['Season','T2_TeamID','T2_seed']

In [299]:
TourneySeeds_T1.head()

,Season,T1_TeamID,T1_seed
0,1998,3330,1
1,1998,3163,2
2,1998,3112,3
3,1998,3301,4
4,1998,3272,5


In [300]:
TourneyDetailedResults = pd.merge(TourneyDetailedResults, TourneySeeds_T1, on = ['Season', 'T1_TeamID'], how = 'left')
TourneyDetailedResults = pd.merge(TourneyDetailedResults, TourneySeeds_T2, on = ['Season', 'T2_TeamID'], how = 'left')

In [301]:
TourneyDetailedResults["Seed_diff"] = TourneyDetailedResults["T1_seed"] - TourneyDetailedResults["T2_seed"]

In [302]:
TourneyDetailedResults.head()

,Season,DayNum,T1_TeamID,T1_Score,T2_TeamID,T2_Score,result,T1_FGM mean,T1_FGA mean,T1_FGM3 mean,T1_FGA3 mean,T1_FTM mean,T1_FTA mean,T1_OR mean,T1_DR mean,T1_Ast mean,T1_TO mean,T1_Stl mean,T1_Blk mean,T1_PF mean,T1_opponent_FGM mean,T1_opponent_FGA mean,T1_opponent_FGM3 mean,T1_opponent_FGA3 mean,T1_opponent_FTM mean,T1_opponent_FTA mean,T1_opponent_OR mean,T1_opponent_DR mean,T1_opponent_Ast mean,T1_opponent_TO mean,T1_opponent_Stl mean,T1_opponent_Blk mean,T1_opponent_PF mean,T1_PointDiff mean,T2_FGM mean,T2_FGA mean,T2_FGM3 mean,T2_FGA3 mean,T2_FTM mean,T2_FTA mean,T2_OR mean,T2_DR mean,T2_Ast mean,T2_TO mean,T2_Stl mean,T2_Blk mean,T2_PF mean,T2_opponent_FGM mean,T2_opponent_FGA mean,T2_opponent_FGM3 mean,T2_opponent_FGA3 mean,T2_opponent_FTM mean,T2_opponent_FTA mean,T2_opponent_OR mean,T2_opponent_DR mean,T2_opponent_Ast mean,T2_opponent_TO mean,T2_opponent_Stl mean,T2_opponent_Blk mean,T2_opponent_PF mean,T2_PointDiff mean,T1_win_ratio_14d,T2_win_ratio_14d,T1_quality,T2_quality,T1_seed,T2_seed,Seed_diff
0,2019,137,3125,52,3376,74,0,26.968750,59.687500,10.43750,29.093750,11.312500,16.468750,11.812500,29.156250,16.312500,12.562500,5.562500,2.125000,14.656250,22.000000,60.437500,6.250000,20.750000,9.343750,13.46875,10.96875,23.312500,11.593750,12.031250,5.625000,2.875000,17.531250,16.093750,27.433333,63.233333,6.000000,18.233333,14.866667,20.533333,13.833333,25.166667,13.100000,13.600000,8.333333,6.300000,16.766667,24.466667,62.133333,4.800000,15.533333,12.766667,17.233333,13.466667,23.300000,11.033333,15.866667,6.766667,3.133333,18.900000,9.233333,1.000000,1.000000,NaN,NaN,13,4,9
1,2019,137,3332,78,3340,40,1,31.937500,63.625000,9.75000,23.343750,11.531250,15.062500,10.937500,26.343750,19.031250,10.125000,6.906250,2.593750,13.000000,24.218750,60.281250,6.656250,21.562500,8.656250,11.68750,10.03125,20.750000,13.687500,13.218750,4.687500,2.500000,14.875000,21.406250,25.967742,60.258065,5.838710,16.806452,12.967742,16.225806,11.064516,28.258065,17.225806,14.741935,9.580645,5.258065,14.935484,22.032258,61.870968,6.548387,21.612903,9.516129,14.290323,13.354839,24.258065,14.548387,15.903226,7.548387,1.806452,16.225806,10.612903,0.666667,0.200000,NaN,NaN,2,15,-13
2,2019,137,3281,77,3179,76,1,22.757576,50.757576,8.30303,22.878788,12.151515,16.424242,9.242424,26.393939,13.969697,16.181818,5.939394,3.181818,17.181818,21.030303,57.787879,4.848485,17.030303,11.030303,15.69697,12.30303,20.242424,8.393939,12.212121,6.757576,1.939394,18.909091,8.030303,28.406250,59.937500,8.750000,24.437500,14.781250,19.343750,11.093750,28.562500,21.093750,17.781250,9.031250,2.562500,17.156250,24.312500,63.156250,7.187500,25.125000,11.250000,15.750000,11.468750,22.281250,13.593750,16.281250,8.656250,2.875000,19.562500,13.281250,0.666667,0.200000,NaN,NaN,7,10,-3
3,2019,137,3280,103,3380,46,1,32.531250,66.156250,5.25000,13.781250,15.781250,21.531250,17.343750,25.468750,16.562500,12.625000,8.656250,5.437500,16.937500,21.093750,55.625000,5.187500,15.562500,10.375000,14.93750,11.03125,18.843750,9.281250,19.656250,5.937500,3.937500,19.968750,28.343750,21.214286,55.750000,4.428571,16.178571,11.750000,18.964286,12.750000,22.857143,11.464286,18.392857,8.642857,2.000000,22.392857,21.071429,50.607143,3.857143,14.392857,15.464286,25.142857,11.535714,24.714286,11.500000,19.321429,8.214286,3.107143,19.285714,-2.857143,1.000000,0.200000,NaN,NaN,1,16,-15
4,2019,137,3276,84,3243,54,1,27.093750,60.437500,4.56250,13.937500,13.218750,18.875000,13.750000,25.656250,15.187500,16.250000,7.312500,3.531250,17.156250,23.156250,57.562500,5.875000,18.875000,11.468750,16.84375,10.53125,21.937500,12.687500,16.625000,7.968750,3.437500,17.750000,8.312500,23.062500,56.718750,5.656250,18.781250,13.031250,18.062500,10.500000,24.437500,13.000000,15.062500,8.000000,4.468750,14.562500,23.031250,60.093750,6.343750,20.156250,11.406250,15.187500,13.375000,24.750000,13.906250,15.937500,8.218750,2.812500,16.500000,1.000000,0.500000,0.333333,NaN,NaN,8,9,-1


In [303]:
features = list(SeasonDetailedResults_Stat_T1.columns[2:999]) + \
    list(SeasonDetailedResults_Stat_T2.columns[2:999]) + \
    list(TourneySeeds_T1.columns[2:999]) + \
    list(TourneySeeds_T2.columns[2:999]) + \
    list(last14days_stats_T1.columns[2:999]) + \
    list(last14days_stats_T2.columns[2:999]) + \
    ["Seed_diff"] + ["T1_quality","T2_quality"]

In [304]:
features

['T1_FGM mean',
 'T1_FGA mean',
 'T1_FGM3 mean',
 'T1_FGA3 mean',
 'T1_FTM mean',
 'T1_FTA mean',
 'T1_OR mean',
 'T1_DR mean',
 'T1_Ast mean',
 'T1_TO mean',
 'T1_Stl mean',
 'T1_Blk mean',
 'T1_PF mean',
 'T1_opponent_FGM mean',
 'T1_opponent_FGA mean',
 'T1_opponent_FGM3 mean',
 'T1_opponent_FGA3 mean',
 'T1_opponent_FTM mean',
 'T1_opponent_FTA mean',
 'T1_opponent_OR mean',
 'T1_opponent_DR mean',
 'T1_opponent_Ast mean',
 'T1_opponent_TO mean',
 'T1_opponent_Stl mean',
 'T1_opponent_Blk mean',
 'T1_opponent_PF mean',
 'T1_PointDiff mean',
 'T2_FGM mean',
 'T2_FGA mean',
 'T2_FGM3 mean',
 'T2_FGA3 mean',
 'T2_FTM mean',
 'T2_FTA mean',
 'T2_OR mean',
 'T2_DR mean',
 'T2_Ast mean',
 'T2_TO mean',
 'T2_Stl mean',
 'T2_Blk mean',
 'T2_PF mean',
 'T2_opponent_FGM mean',
 'T2_opponent_FGA mean',
 'T2_opponent_FGM3 mean',
 'T2_opponent_FGA3 mean',
 'T2_opponent_FTM mean',
 'T2_opponent_FTA mean',
 'T2_opponent_OR mean',
 'T2_opponent_DR mean',
 'T2_opponent_Ast mean',
 'T2_opponent_TO m

In [305]:
# Do same for test

In [248]:
test = test.rename(columns={"WTeamID": "T1_TeamID", "LTeamID": "T2_TeamID"})

In [249]:
test.head()

,ID,Season,T1_TeamID,T2_TeamID
0,2015_3106_3107,2015,3106,3107
1,2015_3106_3110,2015,3106,3110
2,2015_3106_3113,2015,3106,3113
3,2015_3106_3114,2015,3106,3114
4,2015_3106_3116,2015,3106,3116


In [185]:
test = pd.merge(test, SeasonDetailedResults_Stat_T1, on = ['Season', 'T1_TeamID'],how = 'left')
test = pd.merge(test, SeasonDetailedResults_Stat_T2, on = ['Season', 'T2_TeamID'],how = 'left')
test = pd.merge(test, glm_quality_T1, on = ['Season', 'T1_TeamID'],how = 'left')
test = pd.merge(test, glm_quality_T2, on = ['Season', 'T2_TeamID'],how = 'left')
test = pd.merge(test, TourneySeeds_T1, on = ['Season', 'T1_TeamID'],how = 'left')
test = pd.merge(test, TourneySeeds_T2, on = ['Season', 'T2_TeamID'],how = 'left')
test = pd.merge(test, last14days_stats_T1, on = ['Season', 'T1_TeamID'],how = 'left')
test = pd.merge(test, last14days_stats_T2, on = ['Season', 'T2_TeamID'],how = 'left')
test["Seed_diff"] = test["T1_seed"] - test["T2_seed"]
test.head()

,ID,Season,T1_TeamID,T2_TeamID,T1_FGM mean,T1_FGA mean,T1_FGM3 mean,T1_FGA3 mean,T1_FTM mean,T1_FTA mean,T1_OR mean,T1_DR mean,T1_Ast mean,T1_TO mean,T1_Stl mean,T1_Blk mean,T1_PF mean,T1_opponent_FGM mean,T1_opponent_FGA mean,T1_opponent_FGM3 mean,T1_opponent_FGA3 mean,T1_opponent_FTM mean,T1_opponent_FTA mean,T1_opponent_OR mean,T1_opponent_DR mean,T1_opponent_Ast mean,T1_opponent_TO mean,T1_opponent_Stl mean,T1_opponent_Blk mean,T1_opponent_PF mean,T1_PointDiff mean,T2_FGM mean,T2_FGA mean,T2_FGM3 mean,T2_FGA3 mean,T2_FTM mean,T2_FTA mean,T2_OR mean,T2_DR mean,T2_Ast mean,T2_TO mean,T2_Stl mean,T2_Blk mean,T2_PF mean,T2_opponent_FGM mean,T2_opponent_FGA mean,T2_opponent_FGM3 mean,T2_opponent_FGA3 mean,T2_opponent_FTM mean,T2_opponent_FTA mean,T2_opponent_OR mean,T2_opponent_DR mean,T2_opponent_Ast mean,T2_opponent_TO mean,T2_opponent_Stl mean,T2_opponent_Blk mean,T2_opponent_PF mean,T2_PointDiff mean,T1_seed,T2_seed,T1_win_ratio_14d,T2_win_ratio_14d,Seed_diff
0,2015_3106_3107,2015,3106,3107,20.464286,56.821429,4.857143,15.678571,14.642857,23.321429,14.392857,28.571429,11.821429,19.428571,6.857143,4.75,22.25,21.321429,59.25,3.428571,13.928571,15.071429,24.857143,14.714286,26.428571,10.142857,16.678571,9.25,2.464286,20.392857,-0.714286,25.935484,54.322581,2.064516,7.096774,15.064516,21.516129,13.129032,23.161290,15.193548,16.935484,10.741935,1.677419,18.290323,19.451613,49.612903,5.838710,17.483871,10.709677,16.258065,10.064516,19.161290,12.645161,20.096774,7.354839,3.870968,20.032258,13.548387,15,13,0.833333,0.0,2
1,2015_3106_3110,2015,3106,3110,20.464286,56.821429,4.857143,15.678571,14.642857,23.321429,14.392857,28.571429,11.821429,19.428571,6.857143,4.75,22.25,21.321429,59.25,3.428571,13.928571,15.071429,24.857143,14.714286,26.428571,10.142857,16.678571,9.25,2.464286,20.392857,-0.714286,23.687500,53.531250,4.468750,13.781250,11.500000,15.875000,8.593750,24.500000,15.406250,12.906250,6.843750,2.843750,15.781250,20.531250,55.031250,4.750000,15.312500,11.406250,16.218750,12.125000,23.250000,11.312500,14.843750,5.468750,2.625000,16.312500,6.125000,15,14,0.833333,0.0,1
2,2015_3106_3113,2015,3106,3113,20.464286,56.821429,4.857143,15.678571,14.642857,23.321429,14.392857,28.571429,11.821429,19.428571,6.857143,4.75,22.25,21.321429,59.25,3.428571,13.928571,15.071429,24.857143,14.714286,26.428571,10.142857,16.678571,9.25,2.464286,20.392857,-0.714286,25.000000,57.125000,4.750000,13.281250,13.187500,18.593750,13.343750,23.281250,14.656250,13.906250,8.093750,2.187500,15.281250,21.625000,51.875000,2.843750,11.187500,9.531250,13.656250,9.343750,22.000000,9.218750,16.875000,6.468750,3.906250,17.656250,12.312500,15,3,0.833333,0.5,12
3,2015_3106_3114,2015,3106,3114,20.464286,56.821429,4.857143,15.678571,14.642857,23.321429,14.392857,28.571429,11.821429,19.428571,6.857143,4.75,22.25,21.321429,59.25,3.428571,13.928571,15.071429,24.857143,14.714286,26.428571,10.142857,16.678571,9.25,2.464286,20.392857,-0.714286,24.218750,54.281250,4.125000,11.000000,12.718750,19.000000,11.968750,22.875000,13.968750,14.156250,8.562500,2.687500,18.125000,18.593750,50.312500,3.125000,11.281250,10.718750,16.187500,11.718750,21.593750,8.218750,19.718750,5.031250,2.218750,18.531250,14.250000,15,11,0.833333,0.0,4
4,2015_3106_3116,2015,3106,3116,20.464286,56.821429,4.857143,15.678571,14.642857,23.321429,14.392857,28.571429,11.821429,19.428571,6.857143,4.75,22.25,21.321429,59.25,3.428571,13.928571,15.071429,24.857143,14.714286,26.428571,10.142857,16.678571,9.25,2.464286,20.392857,-0.714286,20.766667,55.766667,4.966667,16.366667,13.800000,21.033333,13.366667,26.933333,11.800000,13.400000,6.400000,3.566667,17.233333,20.066667,55.500000,4.033333,14.100000,11.233333,17.133333,11.266667,24.333333,9.833333,13.466667,6.466667,4.766667,18.900000,4.900000,15,10,0.833333,0.5,5


In [186]:
# merge with WSeasons
columns2 = Seasons.columns.difference(TourneyDetailedResults.columns).tolist() + ["Season"]
TourneyDetailedResults = TourneyDetailedResults.merge(Seasons[columns2], how="left", on="Season")

In [187]:
TourneyDetailedResults = TourneyDetailedResults.drop(['RegionW','RegionX','RegionY','RegionZ'], axis=1)

In [188]:
TourneyDetailedResults = TourneyDetailedResults.sort_values(by=['DayZero', 'DayNum']).reset_index(drop=True)

In [189]:
TourneyDetailedResults.head()

,Season,DayNum,T1_TeamID,T1_Score,T2_TeamID,T2_Score,result,T1_FGM mean,T1_FGA mean,T1_FGM3 mean,T1_FGA3 mean,T1_FTM mean,T1_FTA mean,T1_OR mean,T1_DR mean,T1_Ast mean,T1_TO mean,T1_Stl mean,T1_Blk mean,T1_PF mean,T1_opponent_FGM mean,T1_opponent_FGA mean,T1_opponent_FGM3 mean,T1_opponent_FGA3 mean,T1_opponent_FTM mean,T1_opponent_FTA mean,T1_opponent_OR mean,T1_opponent_DR mean,T1_opponent_Ast mean,T1_opponent_TO mean,T1_opponent_Stl mean,T1_opponent_Blk mean,T1_opponent_PF mean,T1_PointDiff mean,T2_FGM mean,T2_FGA mean,T2_FGM3 mean,T2_FGA3 mean,T2_FTM mean,T2_FTA mean,T2_OR mean,T2_DR mean,T2_Ast mean,T2_TO mean,T2_Stl mean,T2_Blk mean,T2_PF mean,T2_opponent_FGM mean,T2_opponent_FGA mean,T2_opponent_FGM3 mean,T2_opponent_FGA3 mean,T2_opponent_FTM mean,T2_opponent_FTA mean,T2_opponent_OR mean,T2_opponent_DR mean,T2_opponent_Ast mean,T2_opponent_TO mean,T2_opponent_Stl mean,T2_opponent_Blk mean,T2_opponent_PF mean,T2_PointDiff mean,T1_win_ratio_14d,T2_win_ratio_14d,T1_quality,T2_quality,T1_seed,T2_seed,Seed_diff,DayZero
0,2018,137,3438,68,3143,62,1,22.967742,56.516129,5.967742,17.354839,10.225806,14.225806,11.548387,25.258065,12.806452,16.161290,6.677419,4.064516,16.161290,22.258065,57.935484,6.225806,20.000000,10.290323,14.838710,12.258065,23.419355,11.580645,14.967742,8.258065,2.838710,15.903226,1.096774,26.612903,58.580645,5.161290,15.354839,10.870968,15.774194,12.451613,25.838710,15.741935,14.903226,5.838710,3.225806,15.290323,24.483871,61.677419,6.645161,20.741935,10.258065,14.161290,11.967742,20.903226,15.096774,12.322581,6.516129,3.096774,14.870968,3.387097,0.500000,0.5,4.778895e+15,2.255687e+15,10,7,3,10/30/2017
1,2018,137,3437,81,3355,74,1,24.600000,57.033333,8.733333,26.566667,8.966667,12.133333,8.300000,26.533333,14.066667,9.700000,4.200000,3.833333,13.300000,22.833333,59.800000,4.633333,16.733333,9.033333,12.600000,11.666667,25.066667,10.733333,9.833333,4.166667,1.600000,14.466667,7.566667,28.064516,61.774194,8.741935,23.806452,13.548387,17.193548,12.322581,28.387097,16.838710,13.129032,7.290323,3.838710,15.129032,22.193548,59.000000,6.677419,21.516129,10.516129,14.967742,10.129032,22.806452,11.709677,14.483871,7.258065,3.000000,17.677419,16.838710,0.000000,0.0,1.725910e+16,1.016626e+16,9,8,1,10/30/2017
2,2018,137,3401,89,3179,76,1,28.090909,61.969697,4.454545,12.666667,16.424242,21.090909,14.575758,27.333333,13.727273,13.484848,6.757576,3.515152,16.000000,25.484848,63.121212,6.484848,20.030303,9.848485,14.030303,11.666667,21.030303,14.818182,13.060606,5.666667,3.393939,17.969697,9.757576,29.500000,60.937500,9.500000,25.562500,13.500000,17.500000,11.843750,27.437500,22.375000,15.500000,8.843750,3.406250,14.843750,25.437500,64.562500,8.031250,25.343750,9.156250,13.218750,13.218750,21.187500,15.125000,15.250000,7.906250,3.343750,17.093750,13.937500,0.666667,0.0,4.108584e+16,8.355541e-02,4,13,-9,10/30/2017
3,2018,137,3397,100,3251,60,1,28.193548,62.870968,4.806452,15.483871,16.290323,23.032258,14.612903,29.419355,15.774194,16.645161,8.645161,4.225806,15.096774,24.741935,65.161290,5.129032,18.032258,9.806452,14.096774,12.354839,23.064516,11.612903,16.193548,8.387097,2.645161,19.032258,13.064516,23.870968,54.870968,4.612903,14.806452,11.096774,16.161290,14.032258,23.322581,13.000000,15.935484,7.935484,3.322581,17.967742,18.677419,49.322581,4.645161,16.225806,12.516129,18.290323,10.000000,19.193548,10.903226,16.258065,7.225806,3.193548,17.580645,8.935484,0.500000,0.0,1.013256e+17,1.454042e-17,3,14,-11,10/30/2017
4,2018,137,3376,63,3299,52,1,28.125000,59.281250,4.562500,13.218750,15.281250,21.812500,12.687500,27.906250,15.031250,13.781250,7.281250,6.406250,15.468750,22.500000,62.000000,4.812500,15.125000,10.750000,15.625000,13.000000,20.531250,11.375000,13.812500,5.906250,3.125000,19.218750,15.531250,23.896552,64.000000,4.413793,17.068966,13.068966,21.586207,16.241379,27.827586,13.689655,18.000000,8.620690,3.793103,20.551724,20.517241,57.724138,5.344828,18.931034,12.965517,19.724138,12.931034,27.827586,12.172414,21.896552

In [ ]:
def concat_row(r):
    if r['T1_TeamID'] < r['T2_TeamID']:
        res = str(r['Season'])+"_"+str(r['T1_TeamID'])+"_"+str(r['T2_TeamID'])
    else:
        res = str(r['Season'])+"_"+str(r['T2_TeamID'])+"_"+str(r['T1_TeamID'])
    return res

# Delete leaked from train
def delete_leaked_from_df_train(df_train, df_test):
    df_train['Concats'] = df_train.apply(concat_row, axis=1)
    df_train_duplicates = df_train[df_train['Concats'].isin(df_test['ID'].unique())]
    df_train_idx = df_train_duplicates.index.values
    df_train = df_train.drop(df_train_idx)
    df_train = df_train.drop('Concats', axis=1)
    
    return df_train 

In [ ]:
TourneyDetailedResults = delete_leaked_from_df_train(TourneyDetailedResults, test)

In [313]:
class BaseModel(object):

    def __init__(self, train_df, test_df, target, features, categoricals=[], 
                n_splits=3, cv_method="KFold", group=None, task="regression", 
                parameter_tuning=False, seed=42, scaler=None, verbose=True):
        self.train_df = train_df
        self.test_df = test_df
        self.target = target
        self.features = features
        self.n_splits = n_splits
        self.categoricals = categoricals
        self.cv_method = cv_method
        self.group = group
        self.task = task
        self.parameter_tuning = parameter_tuning
        self.seed = seed
        self.scaler = scaler
        self.cv = self.get_cv()
        self.verbose = verbose
        self.params = self.get_params()
        self.y_pred, self.score, self.model, self.oof, self.y_val, self.fi_df = self.fit()

    def train_model(self, train_set, val_set):
        raise NotImplementedError

    def get_params(self):
        raise NotImplementedError

    def convert_dataset(self, x_train, y_train, x_val, y_val):
        raise NotImplementedError

    def convert_x(self, x):
        return x

    def calc_metric(self, y_true, y_pred): # this may need to be changed based on the metric of interest
        if self.task == "multiclass":
            return log_loss(y_true, y_pred)
        elif self.task == "binary":
            return log_loss(y_true, y_pred)
#             return roc_auc_score(y_true, y_pred)
        elif self.task == "regression":
            return np.sqrt(mean_squared_error(y_true, y_pred))

    def get_cv(self):
        if self.cv_method == "KFold":
            cv = KFold(n_splits=self.n_splits, shuffle=True, random_state=self.seed)
            return cv.split(self.train_df)
        elif self.cv_method == "StratifiedKFold":
            cv = StratifiedKFold(n_splits=self.n_splits, shuffle=True, random_state=self.seed)
            return cv.split(self.train_df, self.train_df[self.target])
        elif self.cv_method == "TimeSeriesSplit":
            cv = TimeSeriesSplit(max_train_size=None, n_splits=self.n_splits)
            return cv.split(self.train_df)

    def fit(self):
        # initialize
        oof_pred = np.zeros((self.train_df.shape[0], ))
        y_vals = np.zeros((self.train_df.shape[0], ))
        y_pred = np.zeros((self.test_df.shape[0], ))
        if self.group is not None:
            if self.group in self.features:
                self.features.remove(self.group)
            if self.group in self.categoricals:
                self.categoricals.remove(self.group)
        fi = np.zeros((self.n_splits, len(self.features)))

        # scaling, if necessary
        if self.scaler is not None:
            # fill NaN
            numerical_features = [f for f in self.features if f not in self.categoricals]
            self.train_df[numerical_features] = self.train_df[numerical_features].fillna(self.train_df[numerical_features].median())
            self.test_df[numerical_features] = self.test_df[numerical_features].fillna(self.test_df[numerical_features].median())
            self.train_df[self.categoricals] = self.train_df[self.categoricals].fillna(self.train_df[self.categoricals].mode().iloc[0])
            self.test_df[self.categoricals] = self.test_df[self.categoricals].fillna(self.test_df[self.categoricals].mode().iloc[0])

            # scaling
            if self.scaler == "MinMax":
                scaler = MinMaxScaler()
            elif self.scaler == "Standard":
                scaler = StandardScaler()
            df = pd.concat([self.train_df[numerical_features], self.test_df[numerical_features]], ignore_index=True)
            scaler.fit(df[numerical_features])
            x_test = self.test_df.copy()
            x_test[numerical_features] = scaler.transform(x_test[numerical_features])
            x_test = [np.absolute(x_test[i]) for i in self.categoricals] + [x_test[numerical_features]]
        else:
            x_test = self.test_df[self.features]

        # fitting with out of fold
        for fold, (train_idx, val_idx) in enumerate(self.cv):
            # train test split
            x_train, x_val = self.train_df.loc[train_idx, self.features], self.train_df.loc[val_idx, self.features]
            y_train, y_val = self.train_df.loc[train_idx, self.target], self.train_df.loc[val_idx, self.target]

            # fitting & get feature importance
            if self.scaler is not None:
                x_train[numerical_features] = scaler.transform(x_train[numerical_features])
                x_val[numerical_features] = scaler.transform(x_val[numerical_features])
                x_train = [np.absolute(x_train[i]) for i in self.categoricals] + [x_train[numerical_features]]
                x_val = [np.absolute(x_val[i]) for i in self.categoricals] + [x_val[numerical_features]]
            train_set, val_set = self.convert_dataset(x_train, y_train, x_val, y_val)
            model, importance = self.train_model(train_set, val_set)
            fi[fold, :] = importance
            conv_x_val = self.convert_x(x_val)
            y_vals[val_idx] = y_val
            oof_pred[val_idx] = model.predict(conv_x_val).reshape(oof_pred[val_idx].shape)
            x_test = self.convert_x(x_test)
            y_pred += model.predict(x_test).reshape(y_pred.shape) / self.n_splits
            print('Partial score of fold {} is: {}'.format(fold, self.calc_metric(y_val, oof_pred[val_idx])))
        
        # feature importance data frame
        fi_df = pd.DataFrame()
        for n in np.arange(self.n_splits):
            tmp = pd.DataFrame()
            tmp["features"] = self.features
            tmp["importance"] = fi[n, :]
            tmp["fold"] = n
            fi_df = pd.concat([fi_df, tmp], ignore_index=True)
        gfi = fi_df[["features", "importance"]].groupby(["features"]).mean().reset_index()
        fi_df = fi_df.merge(gfi, on="features", how="left", suffixes=('', '_mean'))

        # outputs
        loss_score = self.calc_metric(y_vals, oof_pred)
        if self.verbose:
            print('Our oof loss score is: ', loss_score)
        return y_pred, loss_score, model, oof_pred, y_vals, fi_df

    def plot_feature_importance(self, rank_range=[1, 50]):
        # plot
        fig, ax = plt.subplots(1, 1, figsize=(10, 20))
        sorted_df = self.fi_df.sort_values(by = "importance_mean", ascending=False).reset_index().iloc[self.n_splits * (rank_range[0]-1) : self.n_splits * rank_range[1]]
        sns.barplot(data=sorted_df, x ="importance", y ="features", orient='h')
        ax.set_xlabel("feature importance")
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        return sorted_df

In [314]:
class LgbModel(BaseModel):
    
    def train_model(self, train_set, val_set):
        verbosity = 100 if self.verbose else 0
        model = lgb.train(self.params, train_set, num_boost_round = 5000, valid_sets=[train_set, val_set], verbose_eval=verbosity)
        fi = model.feature_importance(importance_type="gain")
        return model, fi

    def convert_dataset(self, x_train, y_train, x_val, y_val):
        train_set = lgb.Dataset(x_train, y_train, categorical_feature=self.categoricals)
        val_set = lgb.Dataset(x_val, y_val, categorical_feature=self.categoricals)
        return train_set, val_set

    def get_params(self):
        # fast fit parameters
        params = {
          'num_leaves': 127,
          'objective': self.task,
          'min_data_in_leaf': 50,
          'max_depth': -1,
          'learning_rate': 0.005,
          "boosting_type": "gbdt",
          "bagging_seed": 11,
          "verbosity": -1,
          'random_state': 42,
         }

        # list is here: https://lightgbm.readthedocs.io/en/latest/Parameters.html
        if self.task == "regression":
            params["metric"] = "rmse"
        elif self.task == "binary":
            params["metric"] = "binary_logloss" # auc
        elif self.task == "multiclass":
            params["metric"] = "multi_logloss" # cross_entropy, auc_mu
        
        # Bayesian Optimization by Optuna
        if self.parameter_tuning == True:
            # define objective function
            def objective(trial):
                # train, test split
                train_x, test_x, train_y, test_y = train_test_split(self.train_df[self.features], 
                                                                    self.train_df[self.target],
                                                                    test_size=0.3, random_state=self.seed)
                dtrain = lgb.Dataset(train_x, train_y, categorical_feature=self.categoricals)
                dtest = lgb.Dataset(test_x, test_y, categorical_feature=self.categoricals)

                # parameters to be explored
                hyperparams = {'num_leaves': trial.suggest_int('num_leaves', 24, 1024),
                        'boosting_type': 'gbdt',
                        'objective': params["objective"],
                        'metric': params["metric"],
                        'max_depth': trial.suggest_int('max_depth', 4, 30),
                        'min_child_weight': trial.suggest_int('min_child_weight', 1, 20),
                        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
                        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
                        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
                        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
                        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
                        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
                        'early_stopping_rounds': 100
                        }

                # LGB
                model = lgb.train(hyperparams, dtrain, valid_sets=dtest, verbose_eval=500)
                pred = model.predict(test_x)
                if (self.task == "binary") | (self.task == "multiclass"):
                    return log_loss(test_y, pred)
                elif self.task == "regression":
                    return np.sqrt(mean_squared_error(test_y, pred))

            # run optimization
            study = optuna.create_study(direction='minimize')
            study.optimize(objective, n_trials=100)

            print('Number of finished trials: {}'.format(len(study.trials)))
            print('Best trial:')
            trial = study.best_trial
            print('  Value: {}'.format(trial.value))
            print('  Params: ')
            for key, value in trial.params.items():
                print('    {}: {}'.format(key, value))

            params = trial.params

            # lower learning rate for better accuracy
            params["learning_rate"] = 0.001

            # plot history
            plot_optimization_history(study)

        return params

In [315]:
test["Season"].unique()

array([2015, 2016, 2017, 2018, 2019], dtype=int64)

In [316]:
train = TourneyDetailedResults.copy()

In [317]:
train.shape

(1260, 68)

In [318]:
target = 'result'
features = train.columns.values.tolist()
dropcols = [target,'DayNum', 'Season','DayZero','T1_TeamID', 'T2_TeamID','T1_Score','T2_Score']

In [319]:
features = [f for f in features if f not in dropcols]
print(features)

['T1_FGM mean', 'T1_FGA mean', 'T1_FGM3 mean', 'T1_FGA3 mean', 'T1_FTM mean', 'T1_FTA mean', 'T1_OR mean', 'T1_DR mean', 'T1_Ast mean', 'T1_TO mean', 'T1_Stl mean', 'T1_Blk mean', 'T1_PF mean', 'T1_opponent_FGM mean', 'T1_opponent_FGA mean', 'T1_opponent_FGM3 mean', 'T1_opponent_FGA3 mean', 'T1_opponent_FTM mean', 'T1_opponent_FTA mean', 'T1_opponent_OR mean', 'T1_opponent_DR mean', 'T1_opponent_Ast mean', 'T1_opponent_TO mean', 'T1_opponent_Stl mean', 'T1_opponent_Blk mean', 'T1_opponent_PF mean', 'T1_PointDiff mean', 'T2_FGM mean', 'T2_FGA mean', 'T2_FGM3 mean', 'T2_FGA3 mean', 'T2_FTM mean', 'T2_FTA mean', 'T2_OR mean', 'T2_DR mean', 'T2_Ast mean', 'T2_TO mean', 'T2_Stl mean', 'T2_Blk mean', 'T2_PF mean', 'T2_opponent_FGM mean', 'T2_opponent_FGA mean', 'T2_opponent_FGM3 mean', 'T2_opponent_FGA3 mean', 'T2_opponent_FTM mean', 'T2_opponent_FTA mean', 'T2_opponent_OR mean', 'T2_opponent_DR mean', 'T2_opponent_Ast mean', 'T2_opponent_TO mean', 'T2_opponent_Stl mean', 'T2_opponent_Blk me

In [320]:
train.head()

,Season,DayNum,T1_TeamID,T1_Score,T2_TeamID,T2_Score,result,T1_FGM mean,T1_FGA mean,T1_FGM3 mean,T1_FGA3 mean,T1_FTM mean,T1_FTA mean,T1_OR mean,T1_DR mean,T1_Ast mean,T1_TO mean,T1_Stl mean,T1_Blk mean,T1_PF mean,T1_opponent_FGM mean,T1_opponent_FGA mean,T1_opponent_FGM3 mean,T1_opponent_FGA3 mean,T1_opponent_FTM mean,T1_opponent_FTA mean,T1_opponent_OR mean,T1_opponent_DR mean,T1_opponent_Ast mean,T1_opponent_TO mean,T1_opponent_Stl mean,T1_opponent_Blk mean,T1_opponent_PF mean,T1_PointDiff mean,T2_FGM mean,T2_FGA mean,T2_FGM3 mean,T2_FGA3 mean,T2_FTM mean,T2_FTA mean,T2_OR mean,T2_DR mean,T2_Ast mean,T2_TO mean,T2_Stl mean,T2_Blk mean,T2_PF mean,T2_opponent_FGM mean,T2_opponent_FGA mean,T2_opponent_FGM3 mean,T2_opponent_FGA3 mean,T2_opponent_FTM mean,T2_opponent_FTA mean,T2_opponent_OR mean,T2_opponent_DR mean,T2_opponent_Ast mean,T2_opponent_TO mean,T2_opponent_Stl mean,T2_opponent_Blk mean,T2_opponent_PF mean,T2_PointDiff mean,T1_win_ratio_14d,T2_win_ratio_14d,T1_quality,T2_quality,T1_seed,T2_seed,Seed_diff
0,2019,137,3125,52,3376,74,0,26.968750,59.687500,10.43750,29.093750,11.312500,16.468750,11.812500,29.156250,16.312500,12.562500,5.562500,2.125000,14.656250,22.000000,60.437500,6.250000,20.750000,9.343750,13.46875,10.96875,23.312500,11.593750,12.031250,5.625000,2.875000,17.531250,16.093750,27.433333,63.233333,6.000000,18.233333,14.866667,20.533333,13.833333,25.166667,13.100000,13.600000,8.333333,6.300000,16.766667,24.466667,62.133333,4.800000,15.533333,12.766667,17.233333,13.466667,23.300000,11.033333,15.866667,6.766667,3.133333,18.900000,9.233333,1.000000,1.000000,NaN,NaN,13,4,9
1,2019,137,3332,78,3340,40,1,31.937500,63.625000,9.75000,23.343750,11.531250,15.062500,10.937500,26.343750,19.031250,10.125000,6.906250,2.593750,13.000000,24.218750,60.281250,6.656250,21.562500,8.656250,11.68750,10.03125,20.750000,13.687500,13.218750,4.687500,2.500000,14.875000,21.406250,25.967742,60.258065,5.838710,16.806452,12.967742,16.225806,11.064516,28.258065,17.225806,14.741935,9.580645,5.258065,14.935484,22.032258,61.870968,6.548387,21.612903,9.516129,14.290323,13.354839,24.258065,14.548387,15.903226,7.548387,1.806452,16.225806,10.612903,0.666667,0.200000,NaN,NaN,2,15,-13
2,2019,137,3281,77,3179,76,1,22.757576,50.757576,8.30303,22.878788,12.151515,16.424242,9.242424,26.393939,13.969697,16.181818,5.939394,3.181818,17.181818,21.030303,57.787879,4.848485,17.030303,11.030303,15.69697,12.30303,20.242424,8.393939,12.212121,6.757576,1.939394,18.909091,8.030303,28.406250,59.937500,8.750000,24.437500,14.781250,19.343750,11.093750,28.562500,21.093750,17.781250,9.031250,2.562500,17.156250,24.312500,63.156250,7.187500,25.125000,11.250000,15.750000,11.468750,22.281250,13.593750,16.281250,8.656250,2.875000,19.562500,13.281250,0.666667,0.200000,NaN,NaN,7,10,-3
3,2019,137,3280,103,3380,46,1,32.531250,66.156250,5.25000,13.781250,15.781250,21.531250,17.343750,25.468750,16.562500,12.625000,8.656250,5.437500,16.937500,21.093750,55.625000,5.187500,15.562500,10.375000,14.93750,11.03125,18.843750,9.281250,19.656250,5.937500,3.937500,19.968750,28.343750,21.214286,55.750000,4.428571,16.178571,11.750000,18.964286,12.750000,22.857143,11.464286,18.392857,8.642857,2.000000,22.392857,21.071429,50.607143,3.857143,14.392857,15.464286,25.142857,11.535714,24.714286,11.500000,19.321429,8.214286,3.107143,19.285714,-2.857143,1.000000,0.200000,NaN,NaN,1,16,-15
4,2019,137,3276,84,3243,54,1,27.093750,60.437500,4.56250,13.937500,13.218750,18.875000,13.750000,25.656250,15.187500,16.250000,7.312500,3.531250,17.156250,23.156250,57.562500,5.875000,18.875000,11.468750,16.84375,10.53125,21.937500,12.687500,16.625000,7.968750,3.437500,17.750000,8.312500,23.062500,56.718750,5.656250,18.781250,13.031250,18.062500,10.500000,24.437500,13.000000,15.062500,8.000000,4.468750,14.562500,23.031250,60.093750,6.343750,20.156250,11.406250,15.187500,13.375000,24.750000,13.906250,15.937500,8.218750,2.812500,16.500000,1.000000,0.500000,0.333333,NaN,NaN,8,9,-1


In [321]:
test.head()

,ID,Season,WTeamID,LTeamID
0,2015_3106_3107,2015,3106,3107
1,2015_3106_3110,2015,3106,3110
2,2015_3106_3113,2015,3106,3113
3,2015_3106_3114,2015,3106,3114
4,2015_3106_3116,2015,3106,3116


In [322]:
train["Season"].unique()

array([2019, 2018, 2016, 2015, 2017, 2014, 2013, 2011, 2010, 2012],
      dtype=int64)

In [323]:
models = {}
test["Pred"] = 0.5
pred = np.zeros(test.shape[0])

In [74]:
lgbm = LgbModel(train, test, target, features, categoricals=[], n_splits=10, 
                    cv_method="TimeSeriesSplit", group=None, task="binary", scaler=None, verbose=True)

[100]	training's binary_logloss: 0.603652	valid_1's binary_logloss: 0.612027
[200]	training's binary_logloss: 0.546805	valid_1's binary_logloss: 0.576441
[300]	training's binary_logloss: 0.506524	valid_1's binary_logloss: 0.549487
[400]	training's binary_logloss: 0.47729	valid_1's binary_logloss: 0.52886
[500]	training's binary_logloss: 0.455531	valid_1's binary_logloss: 0.515891
[600]	training's binary_logloss: 0.438248	valid_1's binary_logloss: 0.510097
[700]	training's binary_logloss: 0.424102	valid_1's binary_logloss: 0.5065
[800]	training's binary_logloss: 0.412149	valid_1's binary_logloss: 0.50476
[900]	training's binary_logloss: 0.4018	valid_1's binary_logloss: 0.502375
[1000]	training's binary_logloss: 0.392639	valid_1's binary_logloss: 0.500392
[1100]	training's binary_logloss: 0.384483	valid_1's binary_logloss: 0.499054
[1200]	training's binary_logloss: 0.377191	valid_1's binary_logloss: 0.497785
[1300]	training's binary_logloss: 0.370646	valid_1's binary_logloss: 0.496996
[1

KeyboardInterrupt: 

In [324]:
for season in test["Season"].unique():
    print(f"Predicting {season}...")
    lgbm = LgbModel(train.loc[train["Season"] < season, :], test.loc[test["Season"] == season, :], target, features, categoricals=[], n_splits=10, 
                    cv_method="TimeSeriesSplit", group=None, task="binary", scaler=None, verbose=True)
    models[season] = lgbm
    test.loc[test["Season"] == season, "Pred"] = lgbm.y_pred

Predicting 2015...


KeyError: "None of [Index(['T1_FGM mean', 'T1_FGA mean', 'T1_FGM3 mean', 'T1_FGA3 mean',\n       'T1_FTM mean', 'T1_FTA mean', 'T1_OR mean', 'T1_DR mean', 'T1_Ast mean',\n       'T1_TO mean', 'T1_Stl mean', 'T1_Blk mean', 'T1_PF mean',\n       'T1_opponent_FGM mean', 'T1_opponent_FGA mean', 'T1_opponent_FGM3 mean',\n       'T1_opponent_FGA3 mean', 'T1_opponent_FTM mean', 'T1_opponent_FTA mean',\n       'T1_opponent_OR mean', 'T1_opponent_DR mean', 'T1_opponent_Ast mean',\n       'T1_opponent_TO mean', 'T1_opponent_Stl mean', 'T1_opponent_Blk mean',\n       'T1_opponent_PF mean', 'T1_PointDiff mean', 'T2_FGM mean',\n       'T2_FGA mean', 'T2_FGM3 mean', 'T2_FGA3 mean', 'T2_FTM mean',\n       'T2_FTA mean', 'T2_OR mean', 'T2_DR mean', 'T2_Ast mean', 'T2_TO mean',\n       'T2_Stl mean', 'T2_Blk mean', 'T2_PF mean', 'T2_opponent_FGM mean',\n       'T2_opponent_FGA mean', 'T2_opponent_FGM3 mean',\n       'T2_opponent_FGA3 mean', 'T2_opponent_FTM mean', 'T2_opponent_FTA mean',\n       'T2_op

In [ ]:
#XGBOOST?

In [153]:
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

In [154]:
params_xgb = {'colsample_bytree': 0.8,                 
              'learning_rate': 0.001,
              'max_depth': 31,
              'subsample': 1,
              'objective':'binary:logistic',
              'eval_metric':'logloss',
              'min_child_weight':3,
              'gamma':0.25,
              'n_estimators':5000
              }

In [162]:
X_train = train.drop(['Season','T1_TeamID','T2_TeamID','DayNum','T1_Score','T2_Score','DayZero','result'], axis=1)
y_train = train.result

In [163]:
test_df = test.drop(['ID','Season','T1_TeamID','T2_TeamID','Pred'], axis=1)

In [164]:
test_df = test_df[X_train.columns]

In [165]:
NFOLDS = 100
folds = KFold(n_splits=NFOLDS)

columns = X_train.columns
splits = folds.split(X_train, y_train)

y_preds_xgb = np.zeros(test_df.shape[0])
y_oof_xgb = np.zeros(X_train.shape[0])
  
for fold_n, (train_index, valid_index) in enumerate(splits):
    print('Fold:',fold_n+1)
    X_train2, X_valid2 = X_train[columns].iloc[train_index], X_train[columns].iloc[valid_index]
    y_train2, y_valid2 = y_train.iloc[train_index], y_train.iloc[valid_index]
    
    train_set = xgb.DMatrix(X_train2, y_train2)
    val_set = xgb.DMatrix(X_valid2, y_valid2)
    test_set = xgb.DMatrix(test_df)
    
    clf = xgb.train(params_xgb, train_set,num_boost_round=5000, evals=[(train_set, 'train'), (val_set, 'val')], early_stopping_rounds=100, verbose_eval=100)
    
    y_preds_xgb += clf.predict(test_set) / NFOLDS
    
    del X_train2, X_valid2, y_train2, y_valid2
    gc.collect()

Fold: 1
[0]	train-logloss:0.692547	val-logloss:0.692708
Multiple eval metrics have been passed: 'val-logloss' will be used for early stopping.

Will train until val-logloss hasn't improved in 100 rounds.
[100]	train-logloss:0.638934	val-logloss:0.642912
[200]	train-logloss:0.592457	val-logloss:0.602795
[300]	train-logloss:0.552037	val-logloss:0.568086
[400]	train-logloss:0.515899	val-logloss:0.537559
[500]	train-logloss:0.483705	val-logloss:0.511265
[600]	train-logloss:0.454875	val-logloss:0.489368
[700]	train-logloss:0.428735	val-logloss:0.47372
[800]	train-logloss:0.405125	val-logloss:0.460998
[900]	train-logloss:0.383709	val-logloss:0.450587
[1000]	train-logloss:0.36412	val-logloss:0.44037
[1100]	train-logloss:0.346216	val-logloss:0.433046
[1200]	train-logloss:0.329966	val-logloss:0.427181
[1300]	train-logloss:0.315171	val-logloss:0.422101
[1400]	train-logloss:0.301329	val-logloss:0.415972
[1500]	train-logloss:0.288466	val-logloss:0.410323
[1600]	train-logloss:0.276612	val-logloss:0

In [167]:
submission = pd.read_csv(os.path.join(path, 'WSampleSubmissionStage1_2020.csv'))
submission['Pred'] = y_preds_xgb
submission

,ID,Pred
0,2015_3106_3107,0.467229
1,2015_3106_3110,0.530143
2,2015_3106_3113,0.096529
3,2015_3106_3114,0.364995
4,2015_3106_3116,0.367708
...,...,...
10075,2019_3413_3417,0.096838
10076,2019_3413_3460,0.322795
10077,2019_3416_3417,0.210465
10078,2019_3416_3460,0.487034


In [168]:
submission.to_csv('NCAAW_Ensemble.csv', index=False)